# AquaCrop-OSPy: Bridging the gap between research and practice in crop-water modelling


<a href="https://colab.research.google.com/github/thomasdkelly/aquacrop/blob/master/tutorials/AquaCrop_OSPy_Notebook_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

This series of notebooks provides users with an introduction to AquaCrop-OSPy, an open-source Python implementation of the U.N. Food and Agriculture Organization (FAO) AquaCrop model. AquaCrop-OSPy is accompanied by a series of Jupyter notebooks, which guide users interactively through a range of common applications of the model. Only basic Python experience is required, and the notebooks can easily be extended and adapted by users for their own applications and needs.


This notebook series consists of four parts:

1.   <a href=https://colab.research.google.com/github/thomasdkelly/aquacrop/blob/master/tutorials/AquaCrop_OSPy_Notebook_1.ipynb>Running an AquaCrop-OSPy model</a>
2.   <a href=https://colab.research.google.com/github/thomasdkelly/aquacrop/blob/master/tutorials/AquaCrop_OSPy_Notebook_2.ipynb>Estimation of irrigation water demands</a>
3.   <a href=https://colab.research.google.com/github/thomasdkelly/aquacrop/blob/master/tutorials/AquaCrop_OSPy_Notebook_3.ipynb>Optimisation of irrigation management strategies</a>
4.  <a href=https://colab.research.google.com/github/thomasdkelly/aquacrop/blob/master/tutorials/AquaCrop_OSPy_Notebook_4.ipynb>Projection of climate change impacts</a>


# Notebook 4: Irrigation demands under different climate change scenarios

In this notebook, we show how AquaCrop-OSPy can be used to simulate impacts of climate change on crop production and irrigation water demands. Climate change is a major driver of production and water scarcity for agriculture globally, and data generated can thus play an important role in designing effective adaptation measures to support farmers and rural economies.

The example presented here focuses on a case study of maize production in the central United States. Baseline data from this region (1982-2018) is used to generate future climate projections using the LARS weather generator. Projections are generated for the periods (2021-2040, 2041-2060, 2061-2080) and for emissions scenarios RCP4.5 and RCP8.5 based on outputs from the EC-EARTH climate model. 

First install and import AquaCrop-OSPy

In [ ]:

!pip install aquacrop==0.2

from aquacrop.classes import    *
from aquacrop.core import       *
from aquacrop.lars import       *

import numpy as np
import pandas as pd
from google.colab import output
output.clear()

In [ ]:
# # only used for local development

# import sys
# _=[sys.path.append(i) for i in ['.', '..']]

# from aquacrop.classes import    *
# from aquacrop.core import       *
# from aquacrop.lars import       *

# import numpy as np
# import pandas as pd

Given the popularity of using LARS-WG for this type of analyis, AquaCrop-OSPy contains a built in function for reading LARS generated outputs. These outputs consist of daily climate projections of precipitation, minimum and maximum temperature and solar radiation. Using the Hargreaves ETo method these values were used to estimate the reference evapotranspiration, resulting in a weather DataFrame in the format required by AquaCrop-OSPy. 

The cell below imports the baseline data from Champion, Nebraska

In [ ]:
# get baseline lars data
lars_fp=get_filepath('CP.dat')
lars_base=prepare_lars_weather(lars_fp,-1,False,order=['year', 'jday', 'maxTemp', 'minTemp', 'precip','rad',])

In [ ]:
lars_base.head()

For each combination of climate scenario and projection period, read in climate data and save the weather DataFrame.

In [ ]:
rcp_list = [45,85]
yr_list = [2030,2050,2070]
yr_range_list = ['2021-2040','2041-2060','2061-2080']

label_list=[]
wdf_list = []
all_year_list=[]
for yr,yr_range in zip(yr_list,yr_range_list):
    for rcp in rcp_list:
        wdf =prepare_lars_weather(get_filepath(f'CP_EC-EARTH[CP,RCP{rcp},{yr_range}]WG.dat'),
                              yr,True,["simyear","jday","minTemp","maxTemp","precip","rad"])
        wdf_list.append(wdf)
        label_list.append(f'RCP{rcp/10},\n{yr_range}')
        all_year_list.append(yr)

In [ ]:
label_list

In [ ]:
# co2 concentrations for the scenarios listed in order
co2_list = [435,449,487,541,524,677]

The LARS generated output files in this example comprise of 100 years of daily weather data (denoted by the 'simyear' column). To select any one of these years, use the `select_lars_wdf` function.

In [ ]:
wdf_list[0].head()

In [ ]:
select_lars_wdf(wdf_list[0],simyear=3).head()

Now that all the climate data is ready, we can define our crop, soil, initial water content and irrigation management. In this example we will use the AquaCrop default Maize crop in calendar day mode. The reason for using calendar day mode is so that the growing season will be the same length in each scenario. 

We choose a Clay loam soil starting at Field Capacity, and an irrigation policy that irrigates if the soil drops below 70% total available water (essentially meeting full crop-water requirements). 

In [ ]:
crop=CropClass('Maize',PlantingDate='05/01',
               CalendarType=1,Emergence = 6,Senescence=107,
               MaxRooting=108,Maturity=132,HIstart=66,
               Flowering=13,YldForm=61,CDC=0.117,CGC=0.163)
          
soil=SoilClass('ClayLoam')
init_wc = InitWCClass() # default is field capacity
irrmngt=IrrMngtClass(1,SMT=[70]*4)

Run the simulation for the baseline period and save the yields and total irrigation.

In [ ]:
#run for baseline scenario
model=AquaCropModel('1982/05/01','2018/10/30',lars_base,soil,crop,
                    init_wc,IrrMngt=irrmngt)
model.initialize()
model.step(till_termination=True)

In [ ]:
baseline_yields = list(model.Outputs.Final['Yield (tonne/ha)'].values)
baseline_tirrs = list(model.Outputs.Final['Seasonal irrigation (mm)'].values)
baseline_labels = ['Baseline']*len(baseline_tirrs)

Define `run_gen_model` function that runs AquaCrop-OSPy for all 100 generated years of a climate scenario and future period (e.g. 2050 at RCP4.5), saving the yields and total irrigation.

In [ ]:
def run_gen_model(all_wdf,co2conc,year):
    gen_yields=[]
    gen_tirrs=[]
    for i in range(100):
        wdf = select_lars_wdf(all_wdf,i+1)
        
        model=AquaCropModel(f'{year}/05/01',f'{year}/10/30',wdf,soil,crop,
                            InitWCClass(),IrrMngt=irrmngt,CO2conc=co2conc)
        model.initialize()
        model.step(till_termination=True)

        gen_yields.append(model.Outputs.Final['Yield (tonne/ha)'].mean())
        gen_tirrs.append(model.Outputs.Final['Seasonal irrigation (mm)'].mean())

    return gen_yields,gen_tirrs

For each combination of climate scenario and projection period, run AquaCrop-OSPy and save results.

In [ ]:
from tqdm.notebook import tqdm # progress bar

all_ylds = []
all_tirrs = []
all_labels = []
for i in tqdm(range(6)):
    year = all_year_list[i]
    wdf = wdf_list[i]
    co2 = co2_list[i]
    label=label_list[i]

    yld_list,tirr_list = run_gen_model(wdf,co2,year)

    all_ylds.extend(yld_list)
    all_tirrs.extend(tirr_list)
    all_labels.extend([label]*len(yld_list))


Combine projection results with baseline.

In [ ]:
all_ylds = baseline_yields+all_ylds
all_tirrs = baseline_tirrs+all_tirrs
all_labels = baseline_labels+all_labels

In [ ]:
import pandas as pd

df = pd.DataFrame([all_ylds,all_tirrs,all_labels]).T
df.columns = ['Yield','Tirr','Label']
df.head()

Visualize results for all projections and baselines.

In [ ]:
# plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# create figure 
fig,ax=plt.subplots(2,1,figsize=(12,14))

# create box plots
sns.boxplot(data=df,x='Label',y='Yield',ax=ax[0])
sns.boxplot(data=df,x='Label',y='Tirr',ax=ax[1])

# labels and fontsize

ax[0].tick_params(labelsize=15)
ax[0].set_xlabel(' ')
ax[0].set_ylabel('Yield (t/ha)',fontsize=18)

ax[1].tick_params(labelsize=15)
ax[1].set_xlabel(' ')
ax[1].set_ylabel('Total Irrigation (ha-mm)',fontsize=18)

# Appendix: Precipitation and ET changes

In [ ]:
all_precip = []
all_et = []
all_new_labels = []
for year in range(1982,2019):
    #run for baseline scenario
    wdf = lars_base[lars_base.Date>f'{year}-05-01']
    wdf = wdf[wdf.Date<f'{year}-10-31']

    all_precip.extend([wdf.Precipitation.mean()])
    all_et.extend([wdf.ReferenceET.mean()])
    all_new_labels.extend(['baseline'])

In [ ]:
from tqdm.notebook import tqdm # progress bar


for i in tqdm(range(6)):

    year = all_year_list[i]
    wdf = wdf_list[i]
    co2 = co2_list[i]
    label=label_list[i]

    wdf = wdf[wdf.Date>f'{year}-05-01']
    wdf = wdf[wdf.Date<f'{year}-10-31']

    #print(wdf.ReferenceET.mean())

    precip_list = list(wdf.groupby('simyear').mean().Precipitation.values)
    et_list = list(wdf.groupby('simyear').mean().ReferenceET.values)


    all_precip.extend(precip_list)
    all_et.extend(et_list)
    all_new_labels.extend([label]*len(et_list))


In [ ]:
import pandas as pd

df = pd.DataFrame([all_precip,all_et,all_new_labels]).T
df.columns = ['precip','et','Label']
df.head()

In [ ]:
# plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# create figure 
fig,ax=plt.subplots(2,1,figsize=(12,14))

# create box plots
sns.boxplot(data=df,x='Label',y='precip',ax=ax[0])
sns.boxplot(data=df,x='Label',y='et',ax=ax[1])

# labels and fontsize

ax[0].tick_params(labelsize=15)
ax[0].set_xlabel(' ')
ax[0].set_ylabel('Precipitation mm',fontsize=18)

ax[1].tick_params(labelsize=15)
ax[1].set_xlabel(' ')
ax[1].set_ylabel('ETo mm',fontsize=18)